In [2]:
import sys
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis\read")
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis")

import numpy as np
import time
import os

from read_scope_data import read_trc_data
from data_analysis_utils import Photons, hl_envelopes_idx
from process_xray_bdot import get_magnetron_power_data

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.signal import fftconvolve, savgol_filter, find_peaks, find_peaks_cwt

%matplotlib qt

In [4]:
ifn = r"E:\good_data\kapton\20241217\C2--E-ring-p30-kapton-xray-d20ms-00058.trc"
# ifn = r"E:\good_data\He3kA_B250G500G_pl0t20_uw17t47_P24\C3--E-ring-p24-z13-x40-xray--00024.trc"
xray_data, tarr_x = read_trc_data(ifn)

if 'kapton' in ifn:
    threshold = [9, 500]
    min_ts = 4e-5
    d = 0
elif "p24" in ifn:
    threshold = [9, 150]
    min_ts = 1e-6
    d = 1
elif "p30" in ifn:
    threshold = [1, 300]
    min_ts = 1e-6
    d = 1

detector = Photons(tarr_x, xray_data, min_timescale=min_ts, distance_mult=d, tsh_mult=threshold, debug=True)
detector.reduce_pulses()

ifn = r"E:\good_data\kapton\20241217\C2--E-ring-p30-kapton-cam--00058.trc"
I_data, tarr_I = read_trc_data(ifn)

plt.figure()
plt.plot(tarr_I, I_data)
plt.show()

Reading data...
Done
Applying Savitzky-Golay filter...
Analyzing optimal downsample rate...

Analyzing downsample rates (minimum 5.0e+03 samples needed for 4.0e-02 ms features):
Rate  2: Mean error = 3.64e-05, Samples in min_timescale = 2500.0
Rate  5: Mean error = 8.23e-05, Samples in min_timescale = 1000.0
Rate 10: Mean error = 1.36e-04, Samples in min_timescale = 500.0
Rate 20: Mean error = 2.44e-04, Samples in min_timescale = 250.0
Rate 50: Mean error = 3.88e-04, Samples in min_timescale = 100.0

Recommended downsample rate: 2
Downsample rate: 2
Computing baseline...
Using constant baseline: 0.003374
Computing thresholds...
Detecting pulses...
Reading data...
Done


In [5]:
min_threshold = 0.01
max_threshold = 0.5
t_bin = 1
bin_centers, counts = detector.counts_per_bin(t_bin, amplitude_min=min_threshold, amplitude_max=max_threshold)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(bin_centers, counts, 'r', label='Photon Counts')

ax_twin = ax.twinx()
ax_twin.plot(detector.tarr_ds, detector.baseline_subtracted, label='X-ray signal', alpha=0.5)
ax_twin.scatter(detector.pulse_times, detector.pulse_amplitudes, label='Detected Pulses', alpha=0.5, c='purple')
ax_twin.set_yticks([])

ax_twin.axhline(y=min_threshold, color='g', linestyle='--', label='Min Threshold')
ax_twin.axhline(y=max_threshold, color='g', linestyle='--', label='Max Threshold')

plt.xlabel('Time (ms)')
plt.ylabel(f'Counts per {t_bin}ms bin')
plt.grid(True)
plt.legend(loc='upper right')  # Added legend location
plt.tight_layout()  # Added tight layout to prevent label cutoff

In [13]:
time_ms = tarr_x * 1000
filtered_xray_data = savgol_filter(xray_data, window_length=31, polyorder=3)

start_idx = 10000000
end_idx = 10500000

plt.figure(0)
plt.plot(time_ms[start_idx:end_idx], xray_data[start_idx:end_idx], label='Original')
plt.plot(time_ms[start_idx:end_idx], filtered_xray_data[start_idx:end_idx], label='Filtered')
plt.legend(loc='upper right')
plt.show()

In [ ]:
min_ts = 1e-2
downsample_rate = 4

# Create downsampled arrays
time_ds = time_ms[::downsample_rate]
data_ds = filtered_xray_data[::downsample_rate]

# Calculate minimum distance between peaks based on min_timescale
sample_period = (time_ds[1] - time_ds[0])  # Time between samples
min_distance = int(min_ts / sample_period)
print(min_distance)

# Plot comparison
plt.figure(figsize=(10,6))
plt.plot(time_ms, -filtered_xray_data, label='Original')
plt.plot(time_ds, -data_ds, '--',label='Downsampled', alpha=0.5)
plt.xlabel('Time (ms)')
plt.ylabel('Signal (V)')
plt.title(f'Original vs Downsampled Data (rate={downsample_rate})')
plt.legend(loc='upper right')
plt.grid(True)
plt.show()

In [37]:
larr, harr = hl_envelopes_idx(data_ds, dmin=100, dmax=200, split=False)

plt.figure()
plt.plot(time_ds, data_ds)
plt.plot(time_ds[larr], data_ds[larr], label='Lower Envelope')
plt.plot(time_ds[harr], data_ds[harr], label='Upper Envelope')
plt.xlabel('Time (ms)')
plt.ylabel('Signal')
plt.legend(loc='upper right')
plt.show()

In [46]:
noise_sample = data_ds[:int(len(data_ds)*0.001)]
noise_amplitude = (np.max(np.abs(noise_sample)) - np.min(np.abs(noise_sample))) / 2

# Interpolate baseline using upper envelope points
baseline = np.interp(np.arange(len(data_ds)), harr, data_ds[harr]) - noise_amplitude
baseline_subtracted = data_ds #- baseline

plt.figure()
plt.plot(time_ds, data_ds, label='Original')
plt.plot(time_ds, baseline, label='Baseline')
plt.plot(time_ds, baseline_subtracted, label='Baseline Subtracted')
plt.legend(loc='upper right')
plt.show()

In [ ]:
data = baseline_subtracted
# Use first 5% of data as noise sample for threshold determination
noise_sample = data[:int(len(data)*0.05)]
noise_mean = np.mean(noise_sample)
noise_std = np.std(noise_sample)

lower_threshold = noise_mean + 6*noise_std
upper_threshold = noise_mean + 100*noise_std

# Find peaks above lower threshold with minimum distance constraint
peak_indices, _ = find_peaks(data, height=lower_threshold, distance=min_distance)

# Remove peaks that exceed upper threshold and nearby peaks
mask = np.ones(len(peak_indices), dtype=bool)
for i, idx in enumerate(peak_indices):
    if data[idx] > upper_threshold:
        # Find all peaks within min_distance of this large peak
        nearby_mask = np.abs(peak_indices - idx) <= min_distance*20
        mask[nearby_mask] = False

# Apply mask to keep only valid peaks
peak_indices = peak_indices[mask]

# Convert peak indices back to original time coordinates
pulse_times = time_ds[peak_indices]
pulse_amplitudes = data[peak_indices]

plt.figure()
plt.plot(time_ds, data)
plt.axhline(y=lower_threshold, color='g', linestyle='--', label='Lower Threshold')
plt.axhline(y=upper_threshold, color='r', linestyle='--', label='Upper Threshold')
plt.scatter(pulse_times, pulse_amplitudes, color='red')
plt.xlabel('Time (ms)')
plt.ylabel('Signal')
plt.title('Detected Pulses')
plt.legend()
plt.show()


In [5]:
# Create histogram of raw signal data
plt.figure(figsize=(10,6))
plt.hist(xray_data, bins=1000, density=True)
plt.xlabel('Signal Amplitude (V)') 
plt.ylabel('Density')
plt.title('Histogram of X-ray Signal Amplitudes')
plt.grid(True)
plt.show()

In [ ]:
# Load and display analysis results
base_dir = r"E:\good_data\He3kA_B250G500G_pl0t20_uw15t45_P24"
analysis_file = os.path.join(base_dir, 'analysis_results.npy')

if os.path.exists(analysis_file):
    # Load the dictionary
    analysis_dict = np.load(analysis_file, allow_pickle=True).item()
    print(f"Found {len(analysis_dict)} shots in analysis results\n")
    
    # Sort file numbers for consistent display
    file_numbers = sorted(analysis_dict.keys())
    
print(file_numbers)

In [8]:
plt.figure(figsize=(12, 8))
for fn in [f"{i:05d}" for i in range(8,25)]:
    bin_centers, counts, r_arr = analysis_dict[fn]
    mask = counts > 5
    scatter = plt.scatter(bin_centers[mask], r_arr[mask]*100, c=counts[mask], cmap='viridis', 
                        s=100, alpha=0.6)

plt.colorbar(scatter, label='Counts')
plt.xlabel('Time (ms)')
plt.ylabel('Position (cm)')
plt.title(base_dir.split('He')[1])
plt.grid(True)
plt.tight_layout()
plt.show()